In [1]:
import mxnet as mx

from mxnet.symbol import Convolution, Activation, Pooling, Dropout, FullyConnected, var, flatten, SoftmaxOutput

import logging
logging.getLogger().setLevel(logging.WARNING)  # logging to stdout

/home/yannick/bin/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
mnist = mx.test_utils.get_mnist()

In [3]:
batch_size = 128

train_iter = mx.io.NDArrayIter(mnist['train_data'], mnist["train_label"], batch_size, shuffle=True)
val_iter = mx.io.NDArrayIter(mnist['test_data'], mnist["test_label"], batch_size, shuffle=True)

In [4]:
def conv_layer(x, num_filter):
    
    x = Convolution(data=x, kernel=(5,5), num_filter=num_filter)
    x = Activation(data=x, act_type="tanh")
    x = Pooling(data=x, pool_type="max", kernel=(2,2), stride=(2,2))

    return x


def full_layers(x, architecture, act_type="tanh", p=None):
    
    for num_cell in architecture:
        x = FullyConnected(x, num_hidden=num_cell)
        if p is not None:
            x = Dropout(x, p=p)
        x = Activation(x, act_type=act_type)

    return x


def generate_lenet(dropout=None, act_type="tanh", full_geom=[500]):
    
    data = var("data")
    
    x = conv_layer(data, 20)
    x = conv_layer(x, 50)
    x = flatten(x)
    x = full_layers(x, full_geom, act_type=act_type, p=dropout)
    x = FullyConnected(data=x, num_hidden=10)
    x = SoftmaxOutput(data=x, name="softmax")
    
    return x, data

In [5]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from functools import partial
import numpy as np
from datetime import datetime

In [8]:
%%time

paramgrid = {
    'lr': hp.choice('lr', np.logspace(-4, -1.5, num=10, base=10)),
    'momentum': hp.uniform('momentum', 0.86, 1.0),
    'dropout': hp.choice('dropout', [None, 0.5, 0.7]),
    'hiddens': hp.choice('hiddens', [128, 256, 512]),
    'epochs': hp.choice('epochs', [10])
}

best_rslt = -np.Inf
start = datetime.now()

max_evals = 5
trials = Trials


def func(params):

    global best_rslt

    print(params)
    
    lenet, data = generate_lenet(dropout=params["dropout"],
                                 act_type='tanh',
                                 full_geom=[params["hiddens"]])
    
    lenet_model = mx.mod.Module(symbol=lenet, context=mx.gpu())

    lr = params["lr"]
    momentum = params["momentum"]
    epochs = params["epochs"]
    
    print("training") 
    
    lenet_model.fit(train_iter,
                eval_data=val_iter,
                optimizer='sgd',
                optimizer_params={'learning_rate': lr, 
                                  'momentum': momentum},
                eval_metric='acc',
                num_epoch=epochs)
    
    test_iter = mx.io.NDArrayIter(mnist['test_data'], None, batch_size)
    prob = lenet_model.predict(test_iter)
    test_iter = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], batch_size)
    
    acc = mx.metric.Accuracy()
    lenet_model.score(test_iter, acc)
    rslt = acc.get()[1]
    
    if rslt > best_rslt:
        best_rslt = rslt
        print(">> %s :", str(datetime.now()-start))
        print(">> %.4f with %s" % (rslt, str(params)))
    
    return {'loss': -rslt, 'status': STATUS_OK}


def false_min(*params, **paramgrid):
    for p in params:
        print("=>%s" % (str(p)))
    for p in paramgrid.keys():
        print('=> %s => %s' %(p, str(paramgrid[p])))

best = fmin(func, paramgrid, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=max_evals, trials=trials)

TypeError: fmin() missing 1 required positional argument: 'space'

In [7]:
trials.fmin()

TypeError: fmin() missing 5 required positional arguments: 'self', 'fn', 'space', 'algo', and 'max_evals'